#### Steps

- Import data /
- Clean file name /
- Clean columns name
- Create table on db using sql
- Import data to db

In [29]:
import os


In [3]:
# import libraries

import pandas as pd
import numpy as np
import os
import psycopg2

In [31]:
!ls

csv_to_postgres.ipynb  hotel_data


In [85]:
!cd hotel_data && ls


hotel_revenue_2018.csv	hotel_revenue_2020.csv	meal_cost.csv
hotel_revenue_2019.csv	market_segment.csv


In [86]:
ls

csv_to_postgres.ipynb  hotel_data/  hotel_revenue_2020.csv


In [4]:
df1 = pd.read_csv('hotel_data/hotel_revenue_2018.csv')
df2 = pd.read_csv('hotel_data/hotel_revenue_2019.csv')
df3 = pd.read_csv('hotel_data/hotel_revenue_2020.csv')
combined_df = df = pd.concat([df1, df2, df3])
df['revenue'] = (combined_df['stays_in_weekend_nights']+combined_df['stays_in_week_nights'])  * combined_df['adr']
len(combined_df.index)
combined_df.tail()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,revenue
40682,City Hotel,1,1,2020,February,5,1,0,0,1,...,NaN,153.0,0,Transient,0.0,0,1,No-Show,2/1/2020,0.0
40683,City Hotel,1,31,2020,July,29,16,2,0,1,...,NaN,NaN,0,Transient,135.0,0,2,No-Show,7/16/2020,270.0
40684,City Hotel,1,25,2020,May,18,6,2,1,1,...,NaN,450.0,0,Transient,125.0,0,0,No-Show,5/6/2020,375.0
40685,City Hotel,1,6,2020,July,29,17,1,0,1,...,NaN,238.0,0,Transient,65.0,0,0,No-Show,7/17/2020,65.0
40686,City Hotel,1,0,2020,August,31,2,0,2,1,...,NaN,40.0,0,Transient,65.0,0,1,No-Show,8/2/2020,130.0


In [7]:
df1['meal']


0        BB
1        HB
2        BB
3        BB
4        BB
         ..
21991    BB
21992    BB
21993    BB
21994    BB
21995    BB
Name: meal, Length: 21996, dtype: object

In [227]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

df2 = combined_df.groupby(['arrival_date_year', 'hotel'])['revenue'].sum()
df2

arrival_date_year  hotel       
2018               City Hotel      3291708.050
                   Resort Hotel    3526408.510
2019               City Hotel     15115188.120
                   Resort Hotel   10706271.520
2020               City Hotel     10198180.780
                   Resort Hotel    6836598.990
Name: revenue, dtype: float64

In [193]:
# df = pd.read_csv('hotel_data/hotel_revenue_2020.csv')
df = pd.read_csv('hotel_data/meal_cost.csv')

df.head()

,Cost,meal
0,0.00,Undefined
1,12.99,BB
2,17.99,HB
3,21.99,FB
4,35.00,SC


In [174]:
# # rename column to see errors
# # df.rename(index={0: "x", 1: "y", 2: "z"})
# df.rename(columns={"is_canceled": "Is Canceled"}, inplace=True)
# df.head()

#### clean_table_names
- remove capital letters
- replace whitespace with
- replace special characters with _

In [194]:

# clean file name 

file = 'meal_cost.csv'

cln_tbl_name = file.lower().replace(' ', '_').replace('!', '_').replace('#', '_').replace('%', '_') \
                .replace('^', '_').replace('&', '').replace('*', '').replace('-', '_').replace('+ ', '_') \
                .replace('=', '_').replace('@', '').replace(r"\\", '').replace(r'(','').replace(r')', '') \
                .replace(r"/","_").replace(r":","").replace('?', '_')
cln_tbl_name

'meal_cost.csv'

In [195]:
# clean columns name

df.columns = (x.lower().replace(' ', '_').replace('!', '_').replace('#', '_').replace('%', '_') \
                .replace('^', '_').replace('&', '').replace('*', '').replace('-', '_').replace('+ ', '_') \
                .replace('=', '_').replace('@', '').replace(r"\\", '').replace(r'(','').replace(r')', '') \
                .replace(r"/","_").replace(r":","").replace('?', '_') for x in df.columns)
df.columns

Index(['cost', 'meal'], dtype='object')

In [196]:
df.dtypes


cost    float64
meal     object
dtype: object

In [179]:
# convert any column data types
# df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])

In [197]:
df.dtypes


cost    float64
meal     object
dtype: object

In [198]:
# Mapping object types
replacements = {
    "object": "varchar",
    "float64" : "float",
    "int64" : "int",
    "datetime64" : "timestamp",
    "datetime64[ns]" : "timestamp",
    "timedelta64[ns]": "varchar",    
}
replacements

{'object': 'varchar',
 'float64': 'float',
 'int64': 'int',
 'datetime64': 'timestamp',
 'datetime64[ns]': 'timestamp',
 'timedelta64[ns]': 'varchar'}

In [199]:
col_str = ", ".join("{} {}".format(n,d) for(n, d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'cost float, meal varchar'

In [200]:
# open database connection
conn_string = "host=127.0.0.1, dbname='testdb' user='pgadmin' password='postgres' "

#  connect to db
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()
print ("opned db succussfully")

opned db succussfully


In [201]:
# drop tables with same name to prevent conflicts
#  Instaert data to tables

cursor.execute('drop table if exists meal_cost;')


In [202]:
#  create table
cursor.execute("create table meal_cost \
                (cost float, meal varchar)")
print("table created successfully")


table created successfully


In [203]:
# save df as csv
df.to_csv('meal_cost.csv', header=df.columns, index=False, encoding='utf-8')

# open the csv file and save it as an object
my_file = open('meal_cost.csv')
print("file opened in memory")


file opened in memory


In [204]:
# upload csv to db
SQL_STATEMENT = """
    COPY meal_cost FROM STDIN WITH
    CSV
    HEADER
    DELIMITER AS ','  
"""
cursor.copy_expert(sql=SQL_STATEMENT, file=my_file)
print("file copied to db")

file copied to db


In [205]:
# grant table access to public
cursor.execute('grant select on table meal_cost to public')
conn.commit()
cursor.close()
print("content imported to db and connection closed")

content imported to db and connection closed


In [207]:
my_file.close()